In [ ]:
import pandas as pd
from scipy import sparse
ratings = pd.read_csv('/content/ratings.csv')
movies = pd.read_csv('/content/movies.csv')
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
print(ratings.shape)
ratings.head()

(100836, 4)


,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [ ]:
userRatings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
userRatings.head()
print("Before: ",userRatings.shape)
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)
#userRatings.fillna(0, inplace=True)
print("After: ",userRatings.shape)

Before:  (610, 9719)
After:  (610, 2269)


In [ ]:
corrMatrix = userRatings.corr(method='pearson')
corrMatrix.head(100)

title,"'burbs, The (1989)",(500) Days of Summer (2009),10 Cloverfield Lane (2016),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),...,Zack and Miri Make a Porno (2008),Zero Dark Thirty (2012),Zero Effect (1998),Zodiac (2007),Zombieland (2009),Zoolander (2001),Zootopia (2016),eXistenZ (1999),xXx (2002),¡Three Amigos! (1986)
title,,,,,,,,,,,,,,,,,,,,,
"'burbs, The (1989)",1.000000,0.063117,-0.023768,0.143482,0.011998,0.087931,0.224052,0.034223,0.009277,0.008331,...,0.017477,0.032470,0.134701,0.153158,0.101301,0.049897,0.003233,0.187953,0.062174,0.353194
(500) Days of Summer (2009),0.063117,1.000000,0.142471,0.273989,0.193960,0.148903,0.142141,0.159756,0.135486,0.200135,...,0.374515,0.178655,0.068407,0.414585,0.355723,0.252226,0.216007,0.053614,0.241092,0.125905
10 Cloverfield Lane (2016),-0.023768,0.142471,1.000000,-0.005799,0.112396,0.006139,-0.016835,0.031704,-0.024275,0.272943,...,0.242663,0.099059,-0.023477,0.272347,0.241751,0.195054,0.319371,0.177846,0.096638,0.002733
10 Things I Hate About You (1999),0.143482,0.273989,-0.005799,1.000000,0.244670,0.223481,0.211473,0.011784,0.091964,0.043383,...,0.243118,0.104858,0.132460,0.091853,0.158637,0.281934,0.050031,0.121029,0.130813,0.110612
"10,000 BC (2008)",0.011998,0.193960,0.112396,0.244670,1.000000,0.234459,0.119132,0.059187,-0.025882,0.089328,...,0.260261,0.087592,0.094913,0.184521,0.242299,0.240231,0.094773,0.088045,0.203002,0.083518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almost Famous (2000),0.099554,0.209549,0.032088,0.296727,0.134434,0.118628,0.242958,0.079158,0.005092,0.051279,...,0.244619,0.085395,0.072505,0.221259,0.126008,0.362571,0.011577,0.208008,0.186599,0.147413
Along Came Polly (2004),0.027287,0.282426,0.113213,0.193085,0.162678,0.180259,0.112928,0.121704,0.125792,0.124032,...,0.173133,0.160430,0.029076,0.189165,0.166278,0.309183,0.078468,-0.036498,0.231566,0.025928
Along Came a Spider (2001),0.064762,-0.003205,0.016372,0.085365,-0.018241,0.080388,0.094016,-0.016678,0.079375,-0.028052,...,0.115347,0.093774,0.085286,0.150757,0.052144,0.174489,0.014189,0.025327,0.233244,0.043581


In [ ]:
def get_similar(movie_name,rating):
    similar_ratings = corrMatrix[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [ ]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies_list = [] # Create an empty list to store the DataFrames

for movie,rating in romantic_lover:
    similar_movie = get_similar(movie,rating)  # Get similar movies for the current movie
    similar_movies_list.append(similar_movie) # Append the result to the list

# Concatenate all DataFrames in the list into a single DataFrame
similar_movies = pd.concat(similar_movies_list, ignore_index=True)

similar_movies.head(10)

,0
0,2.500000
1,1.254898
2,1.157021
3,1.129961
4,1.123890
5,1.082774
6,1.076731
7,1.050643
8,1.050386
9,1.043048


In [ ]:
import pandas as pd

# ... (your previous code to calculate similar_movies) ...

# Sort the DataFrame by the 'rating' column in descending order
sorted_similar_movies = similar_movies.sort_values(ascending=False)

# Get the top 20 recommendations, include index (movie names)
top_20_recommendations = sorted_similar_movies.head(20)

# Get the original movie names from the index of 'similar_movies'
movie_names = similar_movies.index[top_20_recommendations.index]

# Create a DataFrame with movie names and ratings
recommendations_df = pd.DataFrame({'movie': movie_names, 'rating': top_20_recommendations.values})

# Display the recommendations
print(recommendations_df)

    movie    rating
0    2270  5.000000
1    9083  4.000000
2    4541  4.000000
3       0  2.500000
4    6812  2.000000
5    6813  1.500000
6    2271  1.500000
7       1  1.370454
8       2  1.324846
9       3  1.302965
10      4  1.301220
11      5  1.298310
12      6  1.297114
13      7  1.284405
14      8  1.268557
15      9  1.250917
16     10  1.246629
17     11  1.236494
18     12  1.232572
19     13  1.227395


In [ ]:
action_lover = [("Amazing Spider-Man, The (2012)", 5),
                 ("Mission: Impossible III (2006)", 4),
                 ("Toy Story 3 (2010)", 2),
                 ("2 Fast 2 Furious (Fast and the Furious 2, The) (2003)", 4)]

movie_list_with_ratings = []

for movie, rating in action_lover:
    similar_movie = get_similar(movie, rating)

    # Get the movie titles and their similarity scores
    for movie_title, similarity_score in similar_movie.items():
        movie_list_with_ratings.append(
            {'movie': movie_title, 'rating': similarity_score, 'original_movie': movie, 'original_rating': rating}
        )

# Convert the list of dictionaries to a DataFrame (optional)
movie_df = pd.DataFrame(movie_list_with_ratings)

# Print the list or the DataFrame
print(movie_list_with_ratings[:20])  # Print first 20 items in the list
# Or
print(movie_df.head(20))  # Print first 20 rows of the DataFrame

[{'movie': 'Amazing Spider-Man, The (2012)', 'rating': 2.5, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'Captain America: The First Avenger (2011)', 'rating': 1.3704535040638723, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'Thor (2011)', 'rating': 1.3248455926901104, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'Wreck-It Ralph (2012)', 'rating': 1.3029645147636524, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'Star Trek Into Darkness (2013)', 'rating': 1.3012198861111122, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'The Hunger Games: Mockingjay - Part 1 (2014)', 'rating': 1.2983101262719159, 'original_movie': 'Amazing Spider-Man, The (2012)', 'original_rating': 5}, {'movie': 'The Amazing Spider-Man 2 (2014)', 'rating': 1.2971138884728748, 'original_movie': 'Amazing Spider-Man, The (2012